### Loading necessary libraries

In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
import seaborn as sns
from PIL import Image
import glob

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report

from tensorflow.keras.callbacks import EarlyStopping

import joblib

### Defining files' path

In [3]:
data_dir = '../data/'
model_dir = '../model/'

val_dir = data_dir + 'valid/'
test_dir = data_dir + 'test/'
train_dir = data_dir + 'train/'

model_file = model_dir + 'CNN.joblib'

### Load model if it already exists

In [4]:
if os.path.exists(model_file) :
    cnn = joblib.load('../model/CNN.joblib')
    scaler = joblib.load('../model/CNN_scaler.joblib')
    print('Model exists')

### Define model if it does not already exist

In [9]:
if not os.path.exists(model_file) :
    print('Model does not exists')
    

Model does not exists


### Train the model on the dataset

### Save model

In [ ]:
joblib.dump(cnn, "../model/CNN.joblib")
joblib.dump(scaler, "../model/CNN_scaler.joblib")